In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

# Train and save model

In [2]:
# Load and split data
data = datasets.load_iris()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=6546)

In [3]:
# Train and test RFC 
clf = RandomForestClassifier(n_jobs=-1)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy score:", accuracy_score(y_test, y_pred))

Accuracy score: 0.9473684210526315


# Reload model

In [4]:
# Save model to file
joblib.dump(clf, "iris.model")

['iris.model']

In [5]:
# Load the model from file and test
new_clf = joblib.load("iris.model")
y_pred = new_clf.predict(X_test)

print("Accuracy score loaded:", accuracy_score(y_test, y_pred))

Accuracy score loaded: 0.9473684210526315


# Create Cloud function

In [6]:
# Write Cloud Function code to file
%%writefile main.py
# Cloud function code
import joblib
from flask import abort

def predict(request):
  request_json = request.get_json()
  features = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
  targets = ["setosa", "versicolor", "virginica"]

  clf = joblib.load("iris.model")

  try:
    feature_values = [request_json[val] for val in features]
  except Exception as e:
    return abort(400, "Wrong input")
    
  return dict(predicted_iris_type=targets[clf.predict([feature_values])[0]])

Writing main.py


In [7]:
# Check versions
import sklearn

# Collect versions
versions = []
versions.append("scikit-learn==" + sklearn.__version__)
versions.append("joblib==" + joblib.__version__)
versions = "\n".join(versions)
print(versions)

# Create requirements file for Cloud Function
with open("requirements.txt", "w") as f:
  f.write(versions)

scikit-learn==1.0.2
joblib==1.2.0


In [8]:
# Zip the cloud function code, requirements and the model
!zip cf.zip main.py iris.model requirements.txt

  adding: main.py (deflated 44%)
  adding: iris.model (deflated 88%)
  adding: requirements.txt (deflated 3%)


In [9]:
# Check Python version
!python -V

Python 3.8.16


Now upload the zip file as the source for your Google Cloud Function. Use the above Python version to prevent unexpected behaviour.

# Request Cloud Function

In [10]:
# Request the Cloud Function (replace the cf_url endpoint with your own)
from requests import post
import json

# Cloud Function API endpoint
cf_url = "https://cloud-computing-demo-euevx2rnnq-lm.a.run.app"

# Cloud function request, measure is cm
cf_request = {  
    "sepal_length": 1,  # measure is cm
    "sepal_width": 2,  # measure is cm
    "petal_length": 3,  # measure is cm
    "petal_width": 4,  # measure is cm
}

# Get the data
response = post(cf_url, json=cf_request)

if response.ok:
  print(json.loads(response.content))
else:
  print(response.content)

{'predicted_iris_type': 'virginica'}
